# Week 2 Lab 4 Changes for usage of Ollama, free Search API, local pdf export

Using Serper API for search and Ollama for the agent models as well as exporting the report not to a mail but pdf in the root dir of the notebook.

# ToDos
## Adding Requests and xhtml2pdf
In Agents Root Dir run "uv add Markdown requests xhtml2pdf && uv sync"

## Adding serper API Key to .env
Register for free Serper API Key
Add SERPER_API_KEY=xxx to .env-File


In [ ]:
from agents import Agent, trace, Runner, function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from openai import AsyncOpenAI
import requests
from xhtml2pdf import pisa
import webbrowser
import markdown

In [ ]:
load_dotenv(override=True)

In [ ]:
# Add Ollama client for local Model
OLLAMA_BASE_URL = "http://localhost:11434/v1"  # Default Ollama OpenAI-compatible endpoint
ollama_client = AsyncOpenAI(
    base_url=OLLAMA_BASE_URL,
    api_key="ollama"  # Ollama doesn't require a real API key, but OpenAI client needs something
)

# Add local Ollama model
ollama_model = OpenAIChatCompletionsModel(
    model="qwen3:8b",  # This should match your Ollama model name
    openai_client=ollama_client
)

In [ ]:
def do_search_web(query: str) -> str:
    """Search the web for the given query using Serper API"""
    url = "https://google.serper.dev/search"
    headers = {
        "X-API-Key": os.environ.get("SERPER_API_KEY"),
        "Content-Type": "application/json"
    }
    data = {"q": query}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        results = response.json().get("organic", [])
        search_results = "\n".join([f"{result['title']}: {result['snippet']}" for result in results[:5]])  # Limit to first 5 results
        return search_results
    else:
        return "Failed to retrieve search results"

@function_tool
def search_web(query: str) -> str:
    return do_search_web(query)

In [ ]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[search_web],
    model=ollama_model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    tools=[search_web],
    model=ollama_model,
    output_type=WebSearchPlan,
)


In [ ]:
def do_save_to_pdf(subject: str, html_body: str) -> dict:
    output_filename = "output.pdf"
    with open(output_filename, "w+b") as f:
        pisa_status = pisa.CreatePDF(html_body, dest=f)
    if pisa_status.err:
        return {"status": "error", "message": "Failed to create PDF"}
    webbrowser.open(output_filename)
    return {"status": "success", "file": output_filename}

@function_tool
def save_to_pdf(subject: str, html_body: str) -> dict:
    return do_save_to_pdf(subject, html_body)

In [ ]:
result = do_save_to_pdf("Test Subject", "<h1>Hello, World!</h1><p>This is a test.</p>")
print(result)

In [ ]:
INSTRUCTIONS = """You are able to save a nicely formatted PDFbased on a detailed report.
You will be provided with a detailed report. You should use your tool to save the file, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

printing_agent = Agent(
    name="PrintingAgent",
    instructions=INSTRUCTIONS,
    tools=[save_to_pdf],
    model=ollama_model,
    )

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model=ollama_model,
    output_type=ReportData,
)

In [ ]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Run the web search directly for each item in the search plan """
    return await asyncio.to_thread(do_search_web, item.query)

In [ ]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def save_pdf(report: ReportData):
    """Save the report as a PDF directly (bypass agent to avoid long tool-calling loops)."""
    print("Writing File ...")
    subject = "Research Report"

    # Convert Markdown -> HTML
    body_html = markdown.markdown(report.markdown_report, extensions=["extra"])

    # Minimal print-friendly HTML + CSS (xhtml2pdf-compatible)
    html_body = f"""<html>
  <head>
    <meta charset="utf-8">
    <title>{subject}</title>
    <style>
      @page {{ size: A4; margin: 1in; }}
      body {{ font-family: Helvetica, Arial, sans-serif; font-size: 12pt; line-height: 1.5; color: #222; }}
      h1, h2, h3 {{ color: #111; margin: 0.4em 0 0.2em; }}
      p {{ margin: 0.2em 0 0.6em; }}
      ul, ol {{ margin: 0.2em 0 0.6em 1.2em; }}
      code, pre {{ font-family: Courier, monospace; font-size: 10pt; background: #f6f8fa; padding: 2pt 4pt; }}
      pre {{ white-space: pre-wrap; }}
      table {{ border-collapse: collapse; width: 100%; margin: 0.6em 0; }}
      th, td {{ border: 1px solid #ccc; padding: 6pt; }}
      blockquote {{ border-left: 3pt solid #ccc; margin: 0.6em 0; padding: 0.1em 0 0.1em 0.6em; color: #555; }}
      hr {{ border: 0; border-top: 1px solid #ccc; margin: 1em 0; }}
    </style>
  </head>
  <body>
    <h1>{subject}</h1>
    {body_html}
  </body>
</html>"""

    _ = await asyncio.to_thread(do_save_to_pdf, subject, html_body)
    print("File saved")
    return report

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await save_pdf(report)  
    print("Hooray!")